In [147]:
import random
import csv
import numpy as np
from statistics import mode

In [148]:
def retList(filename):
    f=open(filename,'r')
    dataString = f.readlines()
    data=[]
    for i in range(len(dataString)):
        data.append(dataString[i].split())
    return np.array(data,dtype=float)

def accuracy(label, predicted):
    N=len(predicted)
    crt=0
    for i in range(N):
        if label[i] == predicted[i]:
            crt = crt+1
            if N != 0:
                return 0
            else :
                return crt/N
            

In [149]:
def giniIndex(data):
    pos, neg = 0,0
    for i in data:
        if i[-1] == 0:
            neg = neg + 1
        else :
            pos = pos + 1
    pos = pos/(pos+neg)
    neg = 1- pos

    return 1 - pos*pos - neg*neg
def entropy(data):
    
    lastIndex=len(data[0])-1
    n0,n1=0,0
    for i in data:
        if i[lastIndex]==0:
            n0=n0+1
        else:
            n1=n1+1
    p1=n0/(n1+n0)
    p2=1-p1
    # prunning
    if max(p1,p2) > 0.95:
        return 0
    if p1 != 0:
        p1=-p1*np.log(p1)
    if p2 != 0:
        p2=-p2*np.log(p2)
    return p1+p2


# def entropy(data):
    
#     lastIndex=len(data[0])-1
#     n0,n1=0,0
#     for i in data:
#         if i[lastIndex]==0:
#             n0=n0+1
#                 # tree['threshold'] = 0
#         else:
#             n1=n1+1
#     p1=n0/(n1+n0)
#     p2=1-p1
#     if p1 != 0:
#         p1=-p1*np.log(p1)
#     if p2 != 0:
#         p2=-p2*np.log(p2)
#     return p1+p2

def BestSplit(data, numThreshold, func, featureList):
    infoGain = 0
    Left = None
    Right = None
    Threshold = None
    Feature = None
    

    for feature in featureList:
        allValsinFeature = data[:, feature]
        minVal = np.min(allValsinFeature)
        maxVal = np.max(allValsinFeature)
        Tvals = np.linspace(minVal,maxVal,numThreshold+1,endpoint=False)[1:]

        for threshold in Tvals:
            left=[]
            right= []
            for inputs in data:
                if inputs[feature] < threshold:
                    left.append(inputs)
                else :
                    right.append(inputs)
            if len(left)>0 and len(right)>0:
                d = func(data) - len(left)/len(data)*func(left) - len(left)/len(data)*func(right)
                if d > infoGain:
                    infoGain = d
                    Left =np.array(left)
                    Right = np.array(right)
                    Threshold = threshold
                    Feature = feature
    return {'threshold':Threshold, 'feature':Feature, 'left':Left, 'right':Right}
     

        


In [150]:
class DecisionTree():

    tree ={}

    def learn(self, data, numThreshold, func, featureList):
        tree = {'feature':None, 'value':None, 'threshold':None, 'leftTree':None, 'rightTree':None}

        if len(data)>0:
            if func(data)!=0:
                split = BestSplit(data, numThreshold, func, featureList)
                tree['feature']=split['feature']
                tree['threshold']=split['threshold']
                if tree['feature'] is not None:
                    leftTree = DecisionTree().learn(split['left'],numThreshold,func, featureList)
                    rightTree = DecisionTree().learn(split['right'],numThreshold,func, featureList)
                    tree['leftTree']=leftTree
                    tree['rightTree']=rightTree
                    self.tree =tree
                    return tree
                else:
                    tree['leftTree']=None
                    tree['rightTree']=None
                    labels=list(data[:,-1])
                    tree['value']=max(labels,key=labels.count)
                    self.tree=tree
                    return tree
            else :
                tree['leftTree']=None
                tree['rightTree']=None
                labels=list(data[:,-1])
                tree['value']=max(labels,key=labels.count)
                self.tree
                return tree
    def classify(self, tree, instance):
        if tree['value'] is not None:
            return tree['value']
        else:
            f = instance[tree['feature']]
            if f<tree['threshold']:
                return self.classify(tree['leftTree'],instance)
            else:
                return self.classify(tree['rightTree'],instance)

def runDecisionTree(train, test, numThreshold=None,func=giniIndex, featureList=None):
    if featureList is None:
        featureList = range(len(data[0])-1)

    Tree= DecisionTree()
    tree = Tree.learn(train,numThreshold,func,featureList)
    results=[]
    for instance in test:
        result = Tree.classify(tree,instance[:-1])
        # print(result)
        results.append(result == instance[-1])
    accuracy = float(results.count(True))/float(len(results))
    print(accuracy)


In [151]:
class Forest():
    forest=[]

    def __Build__(self,data, numTrees, numThreshold, func ):
        forest = []
        for i in range(numTrees):
            np.random.shuffle(data)
            n=len(data)
            m=len(data[0])-1
            train = data[:int(n/10)]
            Tree = DecisionTree()
            Tree.learn(train, numThreshold, func, np.random.choice(m,int(m/3)) )
            forest.append(Tree)
        self.forest=forest
        return forest

def runForest(train, test, numTrees=100, numThreshold=10, func=entropy):
    MYFOREST= Forest()
    myforest = MYFOREST.__Build__(train,numTrees,numThreshold,func)
    predicted=[]
    for instance in test:
        results=[]
        for Tree in myforest:
            result = Tree.classify(Tree.tree,instance[:-1])
            results.append(result)
        # print(results)
        predicted.append(mode(results) == instance[-1])
    accuracy = float(predicted.count(True))/float(len(predicted))
    print(accuracy)    
    



In [152]:
data = retList("data.txt")
random.shuffle(data)
n=len(data)
train= data[:int(0.7*n)]
test= data[int(0.7*n):]
print(train.shape,test.shape)
# Tree= DecisionTree()
# tree=Tree.learn(train)
# print(tree)
runDecisionTree(train,test,10,entropy)
runForest(train,test,20,10,entropy)

(3220, 58) (1381, 58)
0.7690079652425779
0.6183924692251991
